# Load and clean Data

## 2024 Turkish scraped Used Card data (df_2024)

In [2]:
import numpy as np
import pandas as pd
import re

# Define the filename of your CSV file
csv_file = '2024_turkey_car_market.csv'

# Load the data from the CSV file into a new DataFrame
df_2024 = pd.read_csv(csv_file)

# Columns 'Paint-changed' and 'Willing to Exchange' are dropped as they are not useful
df_2024.drop(columns=['Boya-değişen', 'Takasa Uygun'], inplace=True)

#############################################
######### Translate df_2024 column headers
#############################################

# Create a dictionary with the old column names as keys and the new column names as values
column_rename_dict = {
    'İlan Tarihi': 'Ad Date',
    'Marka': 'Make',
    'Seri': 'Model',
    'Model': 'Package',
    'Yıl': 'Year',
    'Kilometre': 'Kilometers (km)',
    'Vites Tipi': 'Transmission',
    'Yakıt Tipi': 'Fuel Type',
    'Kasa Tipi': 'Body Type',
    'Renk': 'Color',
    'Motor Hacmi': 'Engine Size (cc)',
    'Motor Gücü': 'Engine Power (hp)',
    'Araç Durumu': 'Vehicle Condition',
    'Ort. Yakıt Tüketimi': 'Avg. Fuel Consumption (L/100km)',
    'Yakıt Deposu': 'Fuel Tank Size (L)',
    'Kimden': 'From',
    'Fiyat': 'Price',
    'Link': 'Link',
    'İlan No': 'Ad Number',
    'Page': 'Page',
    'Çekiş': 'Traction'
}

# Rename the columns using the rename method
df_2024.rename(columns=column_rename_dict, inplace=True)

# Add "Scraped Date"
df_2024['Scraped Date'] = pd.to_datetime('2024-05-26')

# Dictionary to map Turkish month names to their English equivalents
month_translation = {
    'Ocak': 'January',
    'Şubat': 'February',
    'Mart': 'March',
    'Nisan': 'April',
    'Mayıs': 'May',
    'Haziran': 'June',
    'Temmuz': 'July',
    'Ağustos': 'August',
    'Eylül': 'September',
    'Ekim': 'October',
    'Kasım': 'November',
    'Aralık': 'December'
}

# Function to replace Turkish month names with English month names
def translate_month(date_str):
    for turkish_month, english_month in month_translation.items():
        date_str = date_str.replace(turkish_month, english_month)
    return date_str

# Apply the translation function to the 'Ad Date' column
df_2024['Ad Date'] = df_2024['Ad Date'].apply(translate_month)

# Convert the 'Ad Date' column to datetime again after translation
df_2024['Ad Date'] = pd.to_datetime(df_2024['Ad Date'], errors='coerce')

# Function to clean and convert 'Kilometers' column
def clean_kilometers(km_str):
    if isinstance(km_str, str):
        km_str = km_str.replace('.', '').replace(' km', '')
    if km_str != km_str:  # Check if km_str is NaN
        return km_str  # Return NaN as is
    return int(km_str)

# Function to clean and convert 'Avg. Fuel Consumption' and 'Fuel Tank Size (L)' columns
def clean_fuel_data(fuel_str):
    if isinstance(fuel_str, str):
        fuel_str = fuel_str.replace(',', '.').replace(' lt', '')
    return float(fuel_str)

# Function to clean and convert 'Price' column
def clean_price(price_str):
    if isinstance(price_str, str):
        price_str = price_str.replace('.', '').replace(' TL', '')
    return int(price_str)

# Function to clean and convert 'Engine Size (cc)' column
def clean_engine_size(engine_size_str):
    if isinstance(engine_size_str, str):
        # Remove non-numeric characters
        engine_size_str = ''.join(filter(str.isdigit, engine_size_str))
        if engine_size_str:
            return int(engine_size_str)
        else:
            return np.nan  # Replace empty values with NaN
    else:
        return np.nan

# Function to clean and convert 'Engine Power (hp)' column
def clean_engine_power(engine_power_str):
    if isinstance(engine_power_str, str):
        # Extract numerical part
        numerical_part = ''.join(filter(str.isdigit, engine_power_str))
        if numerical_part:
            return int(numerical_part)
        else:
            return np.nan  # Replace empty values with NaN
    else:
        return np.nan

# Function to clean and convert 'Ad Number' column
def clean_ad_number(ad_number_str):
    if isinstance(ad_number_str, str):
        # Extract numeric part
        numeric_part = ''.join(filter(str.isdigit, ad_number_str))
        if numeric_part:
            return int(numeric_part)
        else:
            return np.nan  # Replace empty values with NaN
    else:
        return np.nan

def adjust_engine_power(hp):
    hp_str = str(hp)
    if len(hp_str.split('.')[0]) == 6:
        return float(hp_str[:3])
    elif len(hp_str.split('.')[0]) == 5 or len(hp_str.split('.')[0]) == 4:
        return float(hp_str[:2])
    else:
        return hp  
    
# Define a function to rename BMW models
def rename_bmw_model(model):
    if model.startswith('1'):
        return '1 Series'
    elif model.startswith('2'):
        return '2 Series'
    elif model.startswith('3'):
        return '3 Series'
    elif model.startswith('4'):
        return '4 Series'
    elif model.startswith('5'):
        return '5 Series'
    elif model.startswith('6'):
        return '6 Series'
    elif model.startswith('7'):
        return '7 Series'
    else:
        return model
    
# Define a function to rename Mercedes models
def rename_mercedes_model(model):
    if any(sub in model for sub in ['EQE', 'EQS', 'SLK', 'CLC', 'CLK', 'CLA', 'AMG GT', 'SLC', 'GLA', 'CL', 'EQC', 'GLC', 'Viano', 'Vito']):
        return model
    elif 'AMG' in model:
        return 'AMG'
    elif 'Maybach' in model:
        return 'Maybach'
    elif model.startswith('A'):
        return 'A Class'
    elif model.startswith('B'):
        return 'B Class'
    elif model.startswith('C'):
        return 'C Class'
    elif model.startswith('E'):
        return 'E Class'
    elif model.startswith('G'):
        return 'G Wagen'
    elif model.startswith('S'):
        return 'S Class'
    else:
        return model
  
    
# Function to rename Fiat models starting with '500' to '500'
def rename_fiat_model(model):
    if model.startswith('500'):
        return '500'
    return model

# Function to rename Audi models based on their starting values
def rename_audi_model(model):
    if model.startswith('A3'):
        return 'A3'
    elif model.startswith('A4'):
        return 'A4'
    elif model.startswith('A5'):
        return 'A5'
    elif model.startswith('A6'):
        return 'A6'
    elif model.startswith('A7'):
        return 'A7'
    return model
  
    
    
# Define replacement dictionaries for each column
transmission_replacements = {
    'Düz': 'Manual',
    'Düz Vites': 'Manual',
    'Otomatik': 'Automatic',
    'Otomatik Vites': 'Automatic',
    'Yarı Otomatik': 'Semi-Automatic',
    'Yarı Otomatik Vites': 'Semi-Automatic'
}

fuel_type_replacements = {
    'Dizel': 'Diesel',
    'LPG & Benzin': 'LPG and Gasoline',
    'Benzin/LPG': 'LPG and Gasoline',
    'Benzin': 'Gasoline',
    'Hibrit': 'Hybrid',
    'Elektrik': 'Electric'
}

color_replacements = {
    'Beyaz': 'White',
    'Beyaz ': 'White',
    'Siyah': 'Black',
    'Siyah ': 'Black',
    'Sarı ': 'Yellow',
    'Bal Rengi ': 'Yellow', 
    'Gri': 'Gray',
    'Gri ': 'Gray',
    'Zeytin Gri ': "Gray",
    'Gri (Gümüş)': 'Silver',
    'Gümüş Gri ': 'Silver',
    'Gümüş ': 'Silver',
    'Kırmızı': 'Red',
    'Kırmızı ': 'Red',
    'Füme': 'Smoke',
    'Füme ': 'Smoke',
    'Mavi': 'Blue',
    'Gri (metalik)': 'Metallic Gray',
    'Lacivert': 'Navy Blue',
    'Lacivert ': 'Navy Blue',
    'Mavi (metalik)': 'Metallic Blue',
    "Mavi ": 'Blue',
    'Bordo': 'Dark Red',
    'Bordo ': 'Dark Red',
    'Bej': 'Beige',
    'Bej ': 'Beige',
    'Yeşil': 'Green',
    'Yeşil ': 'Green',
    'Kahverengi': 'Brown',
    'Kahverengi ': 'Brown',
    'Gri (titanyum)': 'Titanium Gray',
    'Yeşil (metalik)': 'Metallic Green',
    'Şampanya': 'Champagne',
    'Şampanya ': 'Champagne',
    'Sarı': 'Yellow',
    'Diğer': 'Other',
    'Diğer ': 'Other',
    'Turkuaz': 'Turquoise',
    'Turkuaz ': 'Turquoise',
    'Turuncu': 'Orange',
    'Turuncu ': 'Orange',
    'Altın': 'Gold',
    'Altın ': 'Gold',
    'Mor': 'Purple',
    'Pembe': 'Pink',
    'Pembe ': 'Pink',
    'Mor ': 'Purple',
    'Kum Rengi': 'Cream',
    'Krem ': 'Cream',
    'Amarant': 'Red',
    'Eflatun ': 'Purple',
    'Ihlamur ': 'Purple'
}


from_replacements = {
    'Sahibinden': 'Owner',
    'Galeriden': 'Dealer',
    'Yetkili Bayiden': 'Manufacturer',
    'Yetkili ': 'Manufacturer',
    'Yetkili': 'Manufacturer'
}

vehicle_condition_replacements = {
    'İkinci El': 'Second Hand',
    '2. El': 'Second Hand',
    'Yurtdışından İthal Sıfır': 'Zero Imported from Abroad',
    'Sıfır': 'New',
    '0 km': "New",
    'Hasarlı': 'Damaged',
    'Klasik': "Classic"
}

traction_replacements = {
    'Önden Çekiş': 'Front-wheel-drive',
    'Arkadan İtiş': 'Rear-wheel-drive',
    '4WD (Sürekli)': '4WD (Continuous)',
    '4x2 (Önden Çekişli)': 'Front-wheel-drive',
    '4x2 (Arkadan İtişli)': 'Rear-wheel-drive',
    'AWD (Elektronik)': 'AWD (Electronic)'
}


body_type_replacements = {
    'Camlı Van': 'Glass Van',
    'Yarım Camlı Van': 'Half Glass Van',
    'Minibüs': 'Minibus',
    'Frigorifik Panelvan': 'Refrigerated Panel Van'
}


# Apply the cleaning functions to the respective columns
df_2024['Kilometers (km)'] = df_2024['Kilometers (km)'].apply(clean_kilometers)
df_2024['Avg. Fuel Consumption (L/100km)'] = df_2024['Avg. Fuel Consumption (L/100km)'].apply(clean_fuel_data)
df_2024['Fuel Tank Size (L)'] = df_2024['Fuel Tank Size (L)'].apply(clean_fuel_data)
df_2024['Price'] = df_2024['Price'].apply(clean_price)
df_2024['Engine Size (cc)'] = df_2024['Engine Size (cc)'].apply(clean_engine_size)
df_2024['Engine Power (hp)'] = df_2024['Engine Power (hp)'].apply(clean_engine_power)
df_2024['Transmission'] = df_2024['Transmission'].replace(transmission_replacements)
df_2024['Fuel Type'] = df_2024['Fuel Type'].replace(fuel_type_replacements)
df_2024['Color'] = df_2024['Color'].replace(color_replacements)
df_2024['From'] = df_2024['From'].replace(from_replacements)
df_2024['Vehicle Condition'] = df_2024['Vehicle Condition'].replace(vehicle_condition_replacements)
df_2024['Traction'] = df_2024['Traction'].replace(traction_replacements)
df_2024['Body Type'] = df_2024['Body Type'].replace(body_type_replacements)
df_2024['Ad Number'] = df_2024['Ad Number'].apply(clean_ad_number)
df_2024['Engine Power (hp)'] = df_2024['Engine Power (hp)'].apply(adjust_engine_power)  
df_2024['Kilometers (km)'] = df_2024['Kilometers (km)'].apply(lambda x: min(x, 950000))
df_2024.loc[df_2024['Make'] == 'BMW', 'Model'] = df_2024.loc[df_2024['Make'] == 'BMW', 'Model'].apply(rename_bmw_model)
df_2024.loc[df_2024['Make'] == 'Mercedes', 'Model'] = df_2024.loc[df_2024['Make'] == 'Mercedes', 'Model'].apply(rename_mercedes_model)    
df_2024.loc[df_2024['Make'] == 'Audi', 'Model'] = df_2024.loc[df_2024['Make'] == 'Audi', 'Model'].apply(rename_audi_model)
df_2024.loc[df_2024['Make'] == 'Fiat', 'Model'] = df_2024.loc[df_2024['Make'] == 'Fiat', 'Model'].apply(rename_fiat_model)
df_2024['Make'] = df_2024['Make'].replace('Mercedes - Benz', 'Mercedes')



###################################################################################################
# Split oldpriceNEWprice
###################################################################################################

# Define a function to extract the new price from the combined price string
def extract_new_price(price_str):
    # Convert price_str to string and remove '.0' if it's present
    price_str = str(price_str).replace('.0', '')
    
    # Check if the price string is long enough to be split
    if len(price_str) >= 12:
        # Split the price into two halves and convert the second half to an integer
        half_length = len(price_str) // 2
        new_price = int(price_str[half_length:])
    else:
        # If the number of digits is less than 12, treat the entire string as the price
        new_price = int(price_str)
    
    return new_price

# Apply the function to create a new "Price" column
df_2024['Price'] = df_2024['Price'].apply(extract_new_price)

# Get the value counts for the "Price" column
price_value_counts = df_2024['Price'].value_counts()

# Manually change some values in the "Price" column
df_2024.at[30523, 'Price'] = 1180000
df_2024.at[44198, 'Price'] = 16000
df_2024.at[37121, 'Price'] = 43000
df_2024.at[4536, 'Price'] = 745000

# Sort the DataFrame by 'Price' in descending order and print the top 20 values
top_20_prices = df_2024.sort_values(by='Price', ascending=False).head(20)


######################################################################################
# Some Engine Size (cc) were intervals and were scraped as huge values
######################################################################################

# Dictionary to map old large engine sizes to their new values
engine_size_replacements = {
    140116003.0: 1600,
    120114003.0: 1400,
    180120003.0: 2000,
    160118003.0: 1800,
    200125003.0: 2500,
    250130003.0: 3000,
    350140003.0: 4000,
    300135003.0: 3500,
    400145003.0: 4500,
    450150003.0: 5000,
    550160003.0: 6000
}

# Replace the values in the DataFrame
df_2024['Engine Size (cc)'] = df_2024['Engine Size (cc)'].replace(engine_size_replacements)


######################################################################################
# there are 1703 NaN in "Engine Size (cc)"" but we can find most of them in "Package"
######################################################################################

# Define the function to extract engine size from the Package
def extract_engine_size(row):
    package_str = row['Package'].strip()  # Remove leading and trailing spaces
    make = row['Make'].strip()  # Remove leading and trailing spaces
    
    if isinstance(package_str, str):
        # Use regex to find engine size patterns like '1.0', '1.25', '2.0', etc., anywhere in the string
        match = re.search(r'\b\d{1,2}\.\d{1,2}\b', package_str)
        if match:
            # Convert to cc (e.g., 1.25 -> 1250)
            return int(float(match.group()) * 1000)
        
        # Handle BMW models separately by looking for three-digit patterns anywhere in the string
        if make == 'BMW':
            match_bmw = re.search(r'\b\d{3}[id]?\b', package_str)
            if match_bmw:
                # Extract the last two digits and convert to cc (e.g., 520 -> 2000)
                bmw_code = match_bmw.group()
                if len(bmw_code) == 3:
                    return int(bmw_code[1:3]) * 100
                elif len(bmw_code) == 4:
                    return int(bmw_code[1:3]) * 100
        
            # Debugging: print rows that should match but don't
            print(f"BMW Package not matched: {package_str}")
        
        # Handle Mercedes models separately by looking for three-digit patterns anywhere in the string
        if make == 'Mercedes':
            match_mercedes = re.search(r'\b\d{3}\b', package_str)
            if match_mercedes:
                # Convert to cc (e.g., 180 -> 1800)
                return int(match_mercedes.group()) * 10

            # Debugging: print rows that should match but don't
            # print(f"Mercedes Package not matched: {package_str}")
    
    return np.nan

# Ensure that all values in the 'Package' and 'Make' columns are treated as strings and stripped of whitespace
df_2024['Package'] = df_2024['Package'].astype(str).str.strip()
df_2024['Make'] = df_2024['Make'].astype(str).str.strip()

# Apply the function to fill in missing engine sizes
df_2024['Engine Size (cc)'] = df_2024.apply(
    lambda row: extract_engine_size(row) if pd.isna(row['Engine Size (cc)']) else row['Engine Size (cc)'],
    axis=1
)


###################################################################################
# there are 1556 NaN in "Engine Size (cc)""; they can be filled by similar: 
# year +- 2yrs / make / model / engine size +- 3% cars highest value count
###################################################################################

######### Ensure that all values in the 'Package' and 'Make' columns are treated as strings and stripped of whitespace
df_2024['Package'] = df_2024['Package'].astype(str).str.strip()
df_2024['Make'] = df_2024['Make'].astype(str).str.strip()

# Apply the function to fill in missing engine sizes
df_2024['Engine Size (cc)'] = df_2024.apply(
    lambda row: extract_engine_size(row) if pd.isna(row['Engine Size (cc)']) else row['Engine Size (cc)'],
    axis=1
)

########## Function to correct 'Engine Size (cc)' values (from ABCD3.0 to ABCD)
def correct_engine_size(engine_size):
    engine_size_str = str(engine_size)
    if engine_size_str.endswith('.0') and len(engine_size_str.split('.')[0]) == 5:
        return int(engine_size_str[:-3])  # Remove the last digit and the ".0"
    return engine_size

# Apply the correction function to the 'Engine Size (cc)' column
df_2024['Engine Size (cc)'] = df_2024['Engine Size (cc)'].apply(correct_engine_size)

# Sort the DataFrame by 'Engine Size (cc)' in descending order
sorted_df = df_2024.sort_values(by='Engine Size (cc)', ascending=False)


############ Define the function to fill missing engine power values with a 3% tolerance on engine size
def fill_missing_engine_power_with_tolerance(row, df, tolerance=0.03):
    if pd.isna(row['Engine Power (hp)']):
        engine_size = row['Engine Size (cc)']
        lower_bound = engine_size * (1 - tolerance)
        upper_bound = engine_size * (1 + tolerance)
        
        # Find similar cars with the same make, model, and engine size within the tolerance range, within a 2-year range
        similar_cars = df[
            (df['Year'].between(row['Year'] - 2, row['Year'] + 2)) &
            (df['Make'] == row['Make']) &
            (df['Model'] == row['Model']) &
            (df['Engine Size (cc)'].between(lower_bound, upper_bound)) &
            (~df['Engine Power (hp)'].isna())
        ]
        
        # If there are similar cars, get the most common engine power value
        if not similar_cars.empty:
            most_common_power = similar_cars['Engine Power (hp)'].mode()
            if not most_common_power.empty:
                return most_common_power.iloc[0]
    
    return row['Engine Power (hp)']

# Apply the function to fill missing engine power values
df_2024['Engine Power (hp)'] = df_2024.apply(lambda row: fill_missing_engine_power_with_tolerance(row, df_2024), axis=1)

# 2 small cars are set as 601 hp; changing them to 60hp
df_2024.at[71961, 'Engine Power (hp)'] = 60
df_2024.at[23977, 'Engine Power (hp)'] = 60
df_2024.at[37704, 'Engine Power (hp)'] = 120
df_2024.at[19089, 'Engine Power (hp)'] = 120


###############################
# 3 cars have Otomobil as Make
###############################

# Filter the DataFrame for rows where 'Make' is 'Otomobil'
otomobil_make = df_2024[df_2024['Make'] == 'Otomobil']

# Define the rows to modify
rows_to_modify = [14655, 14656, 33787]

# Update the DataFrame
for index in rows_to_modify:
    df_2024.at[index, 'Make'] = df_2024.at[index, 'Model']
    df_2024.at[index, 'Model'] = df_2024.at[index, 'Package']

    
    
####################################################################################################
## Drop the 'Vehicle Condition' column as it has only 28 values other than "Second Hand"
####################################################################################################

df_2024.drop(columns=['Vehicle Condition'], inplace=True)

# Removing rows with index values 14659 and 64884
df_2024 = df_2024.drop([14659, 64884])
df_2024.loc[76150, 'Price'] = 50865.00    
    
    
###########################
# NaN
###########################

# List of columns to check for NaN values
columns_to_check = ['Ad Date', 'Model', 'Year', 'Kilometers (km)', 'Engine Size (cc)', 'Engine Power (hp)',
                    'From', 'Link', 'Ad Number', 'Page', 'Traction', 'Scraped Date']

# Remove observations with NaN in any of the specified columns
df_2024 = df_2024.dropna(subset=columns_to_check)

# Remove observations where 'Make' value is 'nan' (string)
df_2024 = df_2024[df_2024['Make'].str.lower() != 'nan']

# Replace NaN values in 'Body Type' based on conditions for 'Model'
df_2024.loc[(df_2024['Model'] == 'AMİ') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/3'
df_2024.loc[(df_2024['Model'] == 'Ducato') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Panel Van'
df_2024.loc[(df_2024['Model'] == 'Jumper') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Panel Van'
df_2024.loc[(df_2024['Model'] == 'Transit') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Panel Van'
df_2024.loc[(df_2024['Model'] == 'Jumpy') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Panel Van'
df_2024.loc[(df_2024['Model'] == 'Crafter') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Panel Van'
df_2024.loc[(df_2024['Model'] == 'Master') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Panel Van'
df_2024.loc[(df_2024['Model'] == 'Transporter') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Panel Van'
df_2024.loc[(df_2024['Model'] == 'Transporter') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Panel Van'
df_2024.loc[(df_2024['Model'] == 'EQE') & (df_2024['Body Type'].isna()), 'Body Type'] = 'SUV'
df_2024.loc[(df_2024['Model'] == 'EQS') & (df_2024['Body Type'].isna()), 'Body Type'] = 'SUV'

# Replace NaN values in 'Body Type' based on conditions for 'Model'
df_2024.loc[(df_2024['Model'] == 'Continental') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Sedan'
df_2024.loc[(df_2024['Model'] == 'C') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Sedan'
df_2024.loc[(df_2024['Model'] == '3 Series') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Sedan'

df_2024.loc[(df_2024['Model'] == 'Astra') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/5'
df_2024.loc[(df_2024['Model'] == 'Cooper') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/5'
df_2024.loc[(df_2024['Model'] == 'Cooper S') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/5'
df_2024.loc[(df_2024['Model'] == '216') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/5'
df_2024.loc[(df_2024['Model'] == 'A3') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/5'
df_2024.loc[(df_2024['Model'] == 'Palio') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/5'
df_2024.loc[(df_2024['Model'] == '307') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/5'

df_2024.loc[(df_2024['Model'] == '206') & (df_2024['Body Type'].isna()), 'Body Type'] = 'Hatchback/3'
df_2024.loc[(df_2024['Model'] == 'Duster') & (df_2024['Body Type'].isna()), 'Body Type'] = 'SUV'

# Replace NaN, 'Other', and '-' in 'Color' column with 'White'
df_2024['Color'] = df_2024['Color'].replace([np.nan, 'Other', '-'], 'White')

#####################################################################################################################
# NaN - Function to fill missing values in 'Avg. Fuel Consumption (L/100km)' and 'Fuel Tank Size (L)' with the most common value from similar cars within a tolerance range
#####################################################################################################################

def fill_missing_value_with_tolerance(row, df, column, tolerance=0.05):
    if pd.isna(row[column]):
        engine_size = row['Engine Size (cc)']
        engine_power = row['Engine Power (hp)']
        lower_bound_size = engine_size * (1 - tolerance)
        upper_bound_size = engine_size * (1 + tolerance)
        lower_bound_power = engine_power * (1 - tolerance)
        upper_bound_power = engine_power * (1 + tolerance)

        # Find similar cars with the same make, model, engine size, and engine power within the tolerance range, within a 2-year range
        similar_cars = df[
            (df['Year'].between(row['Year'] - 2, row['Year'] + 2)) &
            (df['Make'] == row['Make']) &
            (df['Model'] == row['Model']) &
            (df['Engine Size (cc)'].between(lower_bound_size, upper_bound_size)) &
            (df['Engine Power (hp)'].between(lower_bound_power, upper_bound_power)) &
            (~df[column].isna())
        ]

        # If there are similar cars, get the most common value for the specified column
        if not similar_cars.empty:
            most_common_value = similar_cars[column].mode()
            if not most_common_value.empty:
                return most_common_value.iloc[0]

    return row[column]

# Apply the function to fill missing 'Avg. Fuel Consumption (L/100km)' values
df_2024['Avg. Fuel Consumption (L/100km)'] = df_2024.apply(
    lambda row: fill_missing_value_with_tolerance(row, df_2024, 'Avg. Fuel Consumption (L/100km)'), axis=1
)

# Apply the function to fill missing 'Fuel Tank Size (L)' values
df_2024['Fuel Tank Size (L)'] = df_2024.apply(
    lambda row: fill_missing_value_with_tolerance(row, df_2024, 'Fuel Tank Size (L)'), axis=1
)


#####################################################################################################################
# NaN - Extend net to look for similar values to fill in NaN in 'Avg. Fuel Consumption (L/100km)' and 'Fuel Tank Size (L)'
#####################################################################################################################

# Function to fill missing values with the most common value from similar cars within a wider tolerance range
def fill_missing_value_with_wider_tolerance(row, df, column, tolerance=0.10):
    if pd.isna(row[column]):
        engine_size = row['Engine Size (cc)']
        lower_bound_size = engine_size * (1 - tolerance)
        upper_bound_size = engine_size * (1 + tolerance)

        # Find similar cars with the same engine size within the wider tolerance range, ignoring other constraints
        similar_cars = df[
            (df['Engine Size (cc)'].between(lower_bound_size, upper_bound_size)) &
            (~df[column].isna())
        ]

        # If there are similar cars, get the most common value for the specified column
        if not similar_cars.empty:
            most_common_value = similar_cars[column].mode()
            if not most_common_value.empty:
                return most_common_value.iloc[0]

    return row[column]

# Apply the function to fill missing 'Avg. Fuel Consumption (L/100km)' values
df_2024['Avg. Fuel Consumption (L/100km)'] = df_2024.apply(
    lambda row: fill_missing_value_with_wider_tolerance(row, df_2024, 'Avg. Fuel Consumption (L/100km)'), axis=1
)

# Apply the function to fill missing 'Fuel Tank Size (L)' values
df_2024['Fuel Tank Size (L)'] = df_2024.apply(
    lambda row: fill_missing_value_with_wider_tolerance(row, df_2024, 'Fuel Tank Size (L)'), axis=1
)

# drop rows with remaining NaN (282 and 287)
# List of columns to check for NaN values
more_columns_to_check = ['Avg. Fuel Consumption (L/100km)', 'Fuel Tank Size (L)']

# Remove observations with NaN in any of the specified columns
df_2024 = df_2024.dropna(subset=more_columns_to_check)

# Replace both NaN and '-' with 'Front-wheel-drive' in the 'Traction' column
df_2024['Traction'].replace({np.nan: 'Front-wheel-drive', '-': 'Front-wheel-drive'}, inplace=True)

##########################
# Clean "Make" column
##########################

# Rename specific values in the 'Make' column
df_2024['Make'] = df_2024['Make'].replace({
    'Ford - Otosan': 'Ford',
    'Ford Trucks': 'Ford',
    'Mitsubishi - Temsa': 'Mitsubishi'
})

# List of values to remove
makes_to_remove = ['Kuba', 'Arora', 'Falcom', 'SYM', 'RKS', 'Bajaj', 'Mondial', 'New Holland', 'Iveco - Otoyol', 'Yamaha', 'Apec', 'BMC', 'Falcon', 'Temsa', 'DS Automobiles', 'Kymco', 'Cupra', 'Iveco', 'Geely']

# Remove rows with the specified values in the 'Make' column
df_2024 = df_2024[~df_2024['Make'].isin(makes_to_remove)]

# List of models to remove - trucks and/or motorcycles
models_to_remove = ['Dio', 'H100', 'PCX 125', 'cbr 125 R', 'NMax 155', 'Kisbee 50', 'JF 26', 'R 1100 RT', 'Axor', 'Roybus', 'NLR', 'NQR', 'Cerez', 'NSS250 Forza', 'NPR', 'CBR 125 R', 'Bongo', 'Cargo', 'X-ADV', 'NC 750 X DCT', 'FE']

# Remove rows with the specified values in the 'Model' column
df_2024 = df_2024[~df_2024['Model'].isin(models_to_remove)]

# Count the number of NaN values in each column
nan_counts = df_2024.isna().sum()

# Print the number of NaN values in each column
print("Counting NaN Values in Dataframe:")
print(nan_counts)


#############################################################################################
# Add "Price USD" Column - USD Amount at Date scraped: 32.19 YTL = 1 USD, May 24th 2024)
#############################################################################################

# Define the exchange rate for May 24, 2024
exchange_rate = 32.19

# Add a new column 'Price USD' by dividing the 'Price' column by the exchange rate
df_2024['Price USD'] = df_2024['Price'] / exchange_rate

#############################################################################################
# Add "Car Age (Years)" Column - USD Amount at Date scraped: 32.19 YTL = 1 USD, May 24th 2024)
#############################################################################################

# Calculate 'Car Age (Years)' and add it as a new column
df_2024['Car Age (Years)'] = 2024 - df_2024['Year'].astype(int)


##########################################################
# Export df_2024 to '2024_turkey_car_market_clean.csv'
##########################################################

# Export the DataFrame to a CSV file
df_2024.to_csv('2024_turkey_car_market_clean.csv', index=False)

Counting NaN Values in Dataframe:
Ad Date                            0
Make                               0
Model                              0
Package                            0
Year                               0
Kilometers (km)                    0
Transmission                       0
Fuel Type                          0
Body Type                          0
Color                              0
Engine Size (cc)                   0
Engine Power (hp)                  0
Avg. Fuel Consumption (L/100km)    0
Fuel Tank Size (L)                 0
From                               0
Price                              0
Link                               0
Ad Number                          0
Page                               0
Traction                           0
Scraped Date                       0
dtype: int64
     Year  Car Age (Years)
0  2016.0                8
1  2011.0               13
2  2017.0                7
3  2020.0                4
4  2018.0                6
